In [33]:
import os
import requests
from bs4 import BeautifulSoup as bs

In [34]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime, timedelta, date
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import StaleElementReferenceException
import time 
import pandas as pd
from io import StringIO
import re

In [35]:
def move_to_next_page(driver):
    try:
        # 현재 페이지를 나타내는 strong 요소 찾기
        current_page = driver.find_element(By.CSS_SELECTOR, "#paging strong")
        # 다음 a 태그 찾기 (current_page 다음에 나오는 a)
        next_page = current_page.find_element(By.XPATH, "following-sibling::a[1]")
        
        # 다음 페이지가 있으면 클릭
        driver.execute_script("arguments[0].click();", next_page)
        time.sleep(3)  # 페이지가 넘어갈 시간 대기
        print("✅ 다음 페이지로 이동합니다")
        return True
    except Exception as e:
        print("✅ 다음 페이지 없음, 크롤링 종료")
        return False

In [40]:
keyword1 = input("지역을 입력해주세요")
keyword2 = "아파트"

지역을 입력해주세요 경기


맥에서 돌릴 때

In [41]:

options = Options()
options.add_experimental_option("detach", True)
options.add_argument("start-maximized")
options.add_argument("Chrome/135.0.0.0")
options.add_argument("lang=ko_KR")


# --- 드라이버 실행 ---
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
wait = WebDriverWait(driver, 10)

# --- 로그인 ---
driver.get('https://www.tankauction.com/')
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.hand'))).click()
wait.until(EC.element_to_be_clickable((By.ID, 'client_id'))).send_keys('yjj7479')
driver.find_element(By.ID, 'passwd').send_keys('xodzmdhrtus0911!')
time.sleep(2)
driver.find_element(By.CSS_SELECTOR, ".button.btn_white.bigrounded").click()
time.sleep(3)

# --- 경매 검색 ---
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".menuicon_10"))).click()
Select(wait.until(EC.element_to_be_clickable((By.ID, "siCd")))).select_by_visible_text(keyword1)
time.sleep(1)
Select(wait.until(EC.element_to_be_clickable((By.ID, "ctgr")))).select_by_visible_text(keyword2)
time.sleep(1)
Select(wait.until(EC.element_to_be_clickable((By.ID, "minbAmtEnd")))).select_by_visible_text('4억')

# --- 날짜 설정 ---
today = date.today()
after_month = today + timedelta(days=30)
start_str = today.strftime("%Y-%m-%d")
end_str = after_month.strftime("%Y-%m-%d")

wait.until(EC.element_to_be_clickable((By.ID, "bgnDt"))).send_keys(start_str)
wait.until(EC.element_to_be_clickable((By.ID, "endDt"))).send_keys(end_str)

# 면적 설정
bldgSqmEnd_input = wait.until(EC.element_to_be_clickable((By.ID, "bldgSqmEnd")))
bldgSqmEnd_input.clear()
bldgSqmEnd_input.send_keys(85)

# --- 검색 및 리스트 로딩 ---
wait.until(EC.element_to_be_clickable((By.ID, "btnSrch"))).click()
Select(wait.until(EC.element_to_be_clickable((By.ID, "dataSize_s")))).select_by_visible_text('100')
WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#lsTbody > tr.hand')))
WebDriverWait(driver, 10).until(lambda d: d.execute_script("return document.readyState") == "complete")

print("\n🔍 리스트 로딩 완료")

# --- 결과 저장용 ---
result1 = []
cols = ['입찰여부','사건번호', '주소','면적','사용승인연도','감정가','최저가','최대입찰가','입찰가','비용','실거래가','최대낙찰비율','예상 낙찰비율','낙찰비율차','낙찰비율','낙찰가','오늘조회수','누적조회수','입찰일']

# --- 페이지별 데이터 수집 ---
while True:
    # 스크롤 끝까지 내려서 전체 tr 다 로딩되게 만들기
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

    # 새로 로딩된 tr 요소들 다시 수집
    trs = driver.find_elements(By.CSS_SELECTOR, "#lsTbody > tr.hand")
    print(f"🔵 현재 페이지 tr 개수: {len(trs)}개")

    # tr 하나하나 수집
    for idx, tr in enumerate(trs, start=1):
        try:
            tid = tr.find_element(By.CSS_SELECTOR, 'input[id^="Tid_"]').get_attribute("value")
            num = tr.find_element(By.CSS_SELECTOR, ".clear.f15.bold").text
            address = tr.find_element(By.CSS_SELECTOR, f'div#adrs_{tid}').get_attribute("innerText")
            start_price = int(tr.find_element(By.ID, f"apslAmt_{tid}").text.replace(",", ""))
            end_price = int(tr.find_element(By.ID, f"minbAmt_{tid}").text.replace(",", ""))
            the_date = tr.find_element(By.ID, f"bidDt_{tid}").text

            # 팝업 열기
            main_window = driver.current_window_handle
            clickable = tr.find_elements(By.CSS_SELECTOR, 'td[onclick^="cntsViewPN"]')[1]
            driver.execute_script("arguments[0].scrollIntoView(true);", clickable)
            time.sleep(1)
            driver.execute_script("arguments[0].click();", clickable)

            # 팝업 전환
            WebDriverWait(driver, 10).until(lambda d: len(d.window_handles) > 1)
            new_window = [w for w in driver.window_handles if w != main_window][0]
            driver.switch_to.window(new_window)

            # 팝업 데이터 수집
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#lyCnt_object")))
            area = driver.find_element(By.CSS_SELECTOR, '#lyCnt_object > table.Ltbl_list > tbody:nth-child(4) > tr > td:nth-child(3)').text

            hit_info = driver.find_element(By.CSS_SELECTOR,'#hitInfo > span').text
            hit_info2 = driver.find_element(By.CSS_SELECTOR,'#hitInfo > span:nth-child(2)').text
            
            try:
                # "매각사례" 섹션 찾기
                auction_sections = driver.find_elements(By.CSS_SELECTOR, "#lyCnt_add .sectTitle")
                auction_table = None

                for section in auction_sections:
                    title_element = section.find_element(By.CSS_SELECTOR, "dt")
                    title_text = title_element.text.strip()

                    if "매각사례" in title_text:
                        # 찾으면 바로 다음 테이블
                        auction_table = section.find_element(By.XPATH, "following-sibling::table[1]")
                        break

                if auction_table is None:
                    raise Exception("매각사례 테이블을 찾지 못했습니다.")

                rows = auction_table.find_elements(By.TAG_NAME, "tr")

                if len(rows) > 1:
                    first_row = rows[1]
                    cells = first_row.find_elements(By.TAG_NAME, "td")
                    
                    if len(cells) >= 4:
                        raw_text = cells[3].text.strip()
                        # 정규식으로 "숫자.숫자%" 만 필터
                        import re
                        match = re.search(r"\d+\.\d+%", raw_text)
                        if match:
                            average_sale_rate = float(match.group().replace('%',''))
                        else:
                            average_sale_rate = None
                    else:
                        average_sale_rate = None
                else:
                    average_sale_rate = None

                print(f"✅ 예상 낙찰 비율 추출 성공: {average_sale_rate}")

            except Exception as e:
                print(f"⚠️ 예상 낙찰 비율 추출 실패: {e}")
                average_sale_rate = None


            # 실거래가
            try:
                real_price_html = driver.find_element(By.CSS_SELECTOR, "#qtLst > tbody > tr > td > div:nth-child(1)").get_attribute("outerHTML")
                df_real = pd.read_html(StringIO(real_price_html))[0]
                real_price = round(df_real['평균가'].apply(lambda x: int(str(x).replace(",", ""))).mean()*10000)
            except Exception as e:
                print(f"⚠️ {idx}번째 실거래가 추출 실패: {e}")

            # 인테리어 비용
            approval_elements = driver.find_elements(By.CSS_SELECTOR, "span.show_b")
            year = 9999
            for elem in approval_elements:
                text = elem.text.strip()
                if "사용승인일" in text:
                    year = int(text.split(":")[1].split("-")[0])
                    break

            if year < 2010:
                try:
                    size_text = driver.find_element(By.CSS_SELECTOR, "#Tr_4 > td:nth-child(3) > div:nth-child(6) > span").text
                    area_size = int(size_text.split("(")[1].split("평")[0])
                    interior = area_size * 1000000
                except:
                    interior = 3000000
            else:
                interior = 3000000

            expense = (start_price * 0.011) + (start_price * 0.004) + 2000000 + (30000000 * 0.24) + interior + (end_price * 0.6 * 0.05 / 2) + 600000 + 1000000
            # 최대 영업이익을 3천으로 가정
            max_pice = real_price - expense - 10000000
            try: 
                if (end_price + expense + 10000000)< real_price and max_pice/start_price * 100 > float(average_sale_rate.replace("%","")):
                            decision = 1 
                else: 
                    decision = 0
            except:
                decision = None
            
            # 최대낙찰비율
            max_price_rate = max_pice/start_price*100 if start_price != None and max_pice !=None else 0
            
            # 낙찰 비율 차
            difference = average_sale_rate-max_price_rate if average_sale_rate != None and max_price_rate != None else 0

            # 팝업 닫고 메인으로 복귀
            driver.close()
            driver.switch_to.window(main_window)

            # 결과 저장
            result1.append([decision, num, address, area, year, start_price, end_price, max_pice, None, expense, real_price,max_price_rate, average_sale_rate, difference,None, None, hit_info, hit_info2, the_date])
            print(f"✅ {idx}번째 tr 추출 완료")

            time.sleep(2)

        except Exception as e:
            print(f"⚠️ {idx}번째 tr 추출 실패: {e}")

    # 다음 페이지 없으면 break
    if not move_to_next_page(driver):
        print("✅ 다음 페이지 없음, 크롤링 종료")
        break

# --- 결과 저장 ---
df = pd.DataFrame(result1, columns=cols)
df.to_csv(f"/Users/jihyun/Desktop/자료/경매/입찰결정/[{today.year}년{today.month:02d}월]{keyword1}_{keyword2}_경매결과.csv", encoding="utf-8-sig", index=False)
print("✅ 파일 저장 완료")




🔍 리스트 로딩 완료
🔵 현재 페이지 tr 개수: 100개
⚠️ 예상 낙찰 비율 추출 실패: 매각사례 테이블을 찾지 못했습니다.
✅ 1번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 82.08
✅ 2번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 69.93
✅ 3번째 tr 추출 완료
⚠️ 예상 낙찰 비율 추출 실패: 매각사례 테이블을 찾지 못했습니다.
✅ 4번째 tr 추출 완료
⚠️ 예상 낙찰 비율 추출 실패: 매각사례 테이블을 찾지 못했습니다.
✅ 5번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 73.38
✅ 6번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 89.05
✅ 7번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 76.89
⚠️ 8번째 실거래가 추출 실패: cannot convert float NaN to integer
✅ 8번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 88.77
✅ 9번째 tr 추출 완료
⚠️ 예상 낙찰 비율 추출 실패: 매각사례 테이블을 찾지 못했습니다.
✅ 10번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 93.73
✅ 11번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 73.02
✅ 12번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: None
✅ 13번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 96.02
⚠️ 14번째 실거래가 추출 실패: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#qtLst > tbody > tr > td > div:nth-child(1)"}
  (Session info: chrome=125.0.6422.61); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-

In [ ]:
df.to_csv(f"/Users/jihyun/Desktop/자료/경매/입찰결정/[{today.year}년{today.month:02d}월]{keyword1}_{keyword2}_경매결과.csv", encoding="utf-8-sig", index=False)
print("✅ 파일 저장 완료")

In [ ]:
df.to_csv(f"./[{today.year}년{today.month:02d}월]{keyword1}_{keyword2}_경매결과.csv", encoding="utf-8-sig", index=False)
print("✅ 파일 저장 완료")


In [10]:
keyword1 = input("지역을 입력해주세요")
keyword2 = input("주거용과 아파트 중 하나를 입력해주세요")

지역을 입력해주세요서울
주거용과 아파트 중 하나를 입력해주세요아파트


# 윈도우에서 돌릴 때

In [11]:

options = Options()
options.add_experimental_option("detach", True)
options.add_argument("start-maximized")
options.add_argument("Chrome/135.0.0.0")
options.add_argument("lang=ko_KR")


# --- 드라이버 실행 ---
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
wait = WebDriverWait(driver, 10)

# --- 로그인 ---
driver.get('https://www.tankauction.com/')
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.hand'))).click()
wait.until(EC.element_to_be_clickable((By.ID, 'client_id'))).send_keys('yjj7479')
driver.find_element(By.ID, 'passwd').send_keys('xodzmdhrtus0911!')
time.sleep(2)
driver.find_element(By.CSS_SELECTOR, ".button.btn_white.bigrounded").click()
time.sleep(3)

# --- 경매 검색 ---
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".menuicon_10"))).click()
Select(wait.until(EC.element_to_be_clickable((By.ID, "siCd")))).select_by_visible_text(keyword1)
time.sleep(1)
Select(wait.until(EC.element_to_be_clickable((By.ID, "ctgr")))).select_by_visible_text(keyword2)
time.sleep(1)
Select(wait.until(EC.element_to_be_clickable((By.ID, "minbAmtEnd")))).select_by_visible_text('4억')

# --- 날짜 설정 ---
today = date.today()
after_month = today + timedelta(days=30)
start_str = today.strftime("%Y-%m-%d")
end_str = after_month.strftime("%Y-%m-%d")

wait.until(EC.element_to_be_clickable((By.ID, "bgnDt"))).send_keys(start_str)
wait.until(EC.element_to_be_clickable((By.ID, "endDt"))).send_keys(end_str)

# 면적 설정
bldgSqmEnd_input = wait.until(EC.element_to_be_clickable((By.ID, "bldgSqmEnd")))
bldgSqmEnd_input.clear()
bldgSqmEnd_input.send_keys(85)

# --- 검색 및 리스트 로딩 ---
wait.until(EC.element_to_be_clickable((By.ID, "btnSrch"))).click()
Select(wait.until(EC.element_to_be_clickable((By.ID, "dataSize_s")))).select_by_visible_text('100')
WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#lsTbody > tr.hand')))
WebDriverWait(driver, 10).until(lambda d: d.execute_script("return document.readyState") == "complete")

print("\n🔍 리스트 로딩 완료")

# --- 결과 저장용 ---
result1 = []
cols = ['입찰여부','사건번호', '주소','면적','사용승인연도','감정가','최저가','최대입찰가','입찰가','비용','실거래가','최대낙찰비율','예상 낙찰비율','낙찰비율차','낙찰비율','낙찰가','오늘조회수','누적조회수','입찰일']

# --- 페이지별 데이터 수집 ---
while True:
    # 스크롤 끝까지 내려서 전체 tr 다 로딩되게 만들기
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

    # 새로 로딩된 tr 요소들 다시 수집
    trs = driver.find_elements(By.CSS_SELECTOR, "#lsTbody > tr.hand")
    print(f"🔵 현재 페이지 tr 개수: {len(trs)}개")

    # tr 하나하나 수집
    for idx, tr in enumerate(trs, start=1):
        try:
            tid = tr.find_element(By.CSS_SELECTOR, 'input[id^="Tid_"]').get_attribute("value")
            num = tr.find_element(By.CSS_SELECTOR, ".clear.f15.bold").text
            address = tr.find_element(By.CSS_SELECTOR, f'div#adrs_{tid}').get_attribute("innerText")
            start_price = int(tr.find_element(By.ID, f"apslAmt_{tid}").text.replace(",", ""))
            end_price = int(tr.find_element(By.ID, f"minbAmt_{tid}").text.replace(",", ""))
            the_date = tr.find_element(By.ID, f"bidDt_{tid}").text

            # 팝업 열기
            main_window = driver.current_window_handle
            clickable = tr.find_elements(By.CSS_SELECTOR, 'td[onclick^="cntsViewPN"]')[1]
            driver.execute_script("arguments[0].scrollIntoView(true);", clickable)
            time.sleep(1)
            driver.execute_script("arguments[0].click();", clickable)

            # 팝업 전환
            WebDriverWait(driver, 10).until(lambda d: len(d.window_handles) > 1)
            new_window = [w for w in driver.window_handles if w != main_window][0]
            driver.switch_to.window(new_window)

            # 팝업 데이터 수집
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#lyCnt_object")))
            area = driver.find_element(By.CSS_SELECTOR, '#lyCnt_object > table.Ltbl_list > tbody:nth-child(4) > tr > td:nth-child(3)').text

            hit_info = driver.find_element(By.CSS_SELECTOR,'#hitInfo > span').text
            hit_info2 = driver.find_element(By.CSS_SELECTOR,'#hitInfo > span:nth-child(2)').text
            
            try:
                # "매각사례" 섹션 찾기
                auction_sections = driver.find_elements(By.CSS_SELECTOR, "#lyCnt_add .sectTitle")
                auction_table = None

                for section in auction_sections:
                    title_element = section.find_element(By.CSS_SELECTOR, "dt")
                    title_text = title_element.text.strip()

                    if "매각사례" in title_text:
                        # 찾으면 바로 다음 테이블
                        auction_table = section.find_element(By.XPATH, "following-sibling::table[1]")
                        break

                if auction_table is None:
                    raise Exception("매각사례 테이블을 찾지 못했습니다.")

                rows = auction_table.find_elements(By.TAG_NAME, "tr")

                if len(rows) > 1:
                    first_row = rows[1]
                    cells = first_row.find_elements(By.TAG_NAME, "td")
                    
                    if len(cells) >= 4:
                        raw_text = cells[3].text.strip()
                        # 정규식으로 "숫자.숫자%" 만 필터
                        import re
                        match = re.search(r"\d+\.\d+%", raw_text)
                        if match:
                            average_sale_rate = float(match.group().replace('%',''))
                        else:
                            average_sale_rate = None
                    else:
                        average_sale_rate = None
                else:
                    average_sale_rate = None

                print(f"✅ 예상 낙찰 비율 추출 성공: {average_sale_rate}")

            except Exception as e:
                print(f"⚠️ 예상 낙찰 비율 추출 실패: {e}")
                average_sale_rate = None


            # 실거래가
            try:
                real_price_html = driver.find_element(By.CSS_SELECTOR, "#qtLst > tbody > tr > td > div:nth-child(1)").get_attribute("outerHTML")
                df_real = pd.read_html(StringIO(real_price_html))[0]
                real_price = round(df_real['평균가'].apply(lambda x: int(str(x).replace(",", ""))).mean()*10000)
            except:
                real_price = 0

            # 인테리어 비용
            approval_elements = driver.find_elements(By.CSS_SELECTOR, "span.show_b")
            year = 9999
            for elem in approval_elements:
                text = elem.text.strip()
                if "사용승인일" in text:
                    year = int(text.split(":")[1].split("-")[0])
                    break

            if year < 2010:
                try:
                    size_text = driver.find_element(By.CSS_SELECTOR, "#Tr_4 > td:nth-child(3) > div:nth-child(6) > span").text
                    area_size = int(size_text.split("(")[1].split("평")[0])
                    interior = area_size * 1000000
                except:
                    interior = 3000000
            else:
                interior = 3000000

            expense = (start_price * 0.011) + (start_price * 0.004) + 2000000 + (30000000 * 0.24) + interior + (end_price * 0.6 * 0.05 / 2) + 600000 + 1000000
            # 최대 영업이익을 3천으로 가정
            max_pice = real_price - expense - 10000000
            try: 
                if (end_price + expense + 10000000)< real_price and max_pice/start_price * 100 > float(average_sale_rate.replace("%","")):
                            decision = 1 
                else: 
                    decision = 0
            except:
                decision = None
            
            # 최대낙찰비율
            max_price_rate = max_pice/start_price if start_price != None and max_pice !=None else 0
            
            # 낙찰 비율 차
            average_sale_rate = average_sale_rate-max_price_rate if average_sale_rate != None and max_price_rate != None else 0

            # 팝업 닫고 메인으로 복귀
            driver.close()
            driver.switch_to.window(main_window)

            # 결과 저장
            result1.append([decision, num, address, area, year, start_price, end_price, max_pice, None, expense, real_price,max_price_rate, average_sale_rate, average_sale_rate,None, None, hit_info, hit_info2, the_date])
            print(f"✅ {idx}번째 tr 추출 완료")

            time.sleep(2)

        except Exception as e:
            print(f"⚠️ {idx}번째 tr 추출 실패: {e}")

    # 다음 페이지 없으면 break
    if not move_to_next_page(driver):
        print("✅ 다음 페이지 없음, 크롤링 종료")
        break

# --- 결과 저장 ---
df = pd.DataFrame(result1, columns=cols)
df.to_csv(f"./[{today.year}년{today.month:02d}월]{keyword1}_{keyword2}_경매결과.csv", encoding="utf-8-sig", index=False)
print("✅ 파일 저장 완료")




🔍 리스트 로딩 완료
🔵 현재 페이지 tr 개수: 53개
✅ 예상 낙찰 비율 추출 성공: 100.03
✅ 1번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: None
✅ 2번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 84.03
✅ 3번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 81.79
✅ 4번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 80.74
✅ 5번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 89.51
✅ 6번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 87.21
✅ 7번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 84.19
✅ 8번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 87.21
✅ 9번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 102.96
✅ 10번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 67.93
✅ 11번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 84.26
✅ 12번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 69.42
✅ 13번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 93.12
✅ 14번째 tr 추출 완료
⚠️ 예상 낙찰 비율 추출 실패: 매각사례 테이블을 찾지 못했습니다.
✅ 15번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 90.23
✅ 16번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 65.38
✅ 17번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 93.12
✅ 18번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 91.43
✅ 19번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 64.38
✅ 20번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 88.62
✅ 21번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 90.41
✅ 22번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 90.95
✅ 23번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 64.38
✅ 24번째 tr 추출 완료


* 주차대수 다시 찾아야함
* 물건번호가 여러개인경우 고려

In [ ]:
#bldg_table > tr:nth-child(4) > td:nth-child(6)

In [20]:
text = "보:225,000,000원"

In [25]:
int(text.replace(",","").split(":")[1].split("원")[0])

225000000

In [16]:
start = 544000000
end = 380800000

In [12]:
area = 25

In [9]:
(274000000 * 0.011) + (274000000 * 0.004) + 2000000 + (30000000 * 0.24) + 25000000 + (274000000 * 0.6 * 0.05 / 2) + 600000 + 1000000

44020000.0

In [ ]:
3000000

In [17]:
(start * 0.011) + (start * 0.004) + 2000000 + ((30000000) * 0.24) + (area*1000000) + (end * 0.6 * 0.05 / 2) + 600000 + 1000000

39672000.0

In [ ]:
if year < 2010:
                try:
                    size_text = driver.find_element(By.CSS_SELECTOR, "#Tr_4 > td:nth-child(3) > div:nth-child(6) > span").text
                    area_size = int(size_text.split("(")[1].split("평")[0])
                    interior = area_size * 1000000
                except:
                    interior = 3000000
            else:
                interior = 3000000

# 낙찰가 가져오기

In [12]:
str_date = input("시작일을 입력해 주세요")
end_date = input("마지막일자를 입력해 주세요")

시작일을 입력해 주세요 20250428
마지막일자를 입력해 주세요 20250430


In [10]:
keyword1 = input("지역을 입력해주세요")
keyword2 = input("주거용과 아파트 중 하나를 입력해주세요")

지역을 입력해주세요 경기
주거용과 아파트 중 하나를 입력해주세요 아파트


In [20]:

options = Options()
options.add_experimental_option("detach", True)
options.add_argument("start-maximized")
options.add_argument("Chrome/135.0.0.0")
options.add_argument("lang=ko_KR")


# --- 드라이버 실행 ---
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
wait = WebDriverWait(driver, 10)

# --- 로그인 ---
driver.get('https://www.tankauction.com/')
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.hand'))).click()
wait.until(EC.element_to_be_clickable((By.ID, 'client_id'))).send_keys('yjj7479')
driver.find_element(By.ID, 'passwd').send_keys('xodzmdhrtus0911!')
time.sleep(2)
driver.find_element(By.CSS_SELECTOR, ".button.btn_white.bigrounded").click()
time.sleep(3)

# --- 경매 검색 ---
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".menuicon_10"))).click()
Select(wait.until(EC.element_to_be_clickable((By.ID, "siCd")))).select_by_visible_text(keyword1)
time.sleep(1)
Select(wait.until(EC.element_to_be_clickable((By.ID, "ctgr")))).select_by_visible_text(keyword2)
time.sleep(1)
Select(wait.until(EC.element_to_be_clickable((By.ID, "minbAmtEnd")))).select_by_visible_text('4억')

# --- 날짜 설정 ---

# str → datetime
start_dt = datetime.strptime(str_date, "%Y%m%d")
end_dt = datetime.strptime(end_date, "%Y%m%d")

# datetime → 원하는 형식의 str
start_str = start_dt.strftime("%Y-%m-%d")
end_str = end_dt.strftime("%Y-%m-%d")


wait.until(EC.element_to_be_clickable((By.ID, "bgnDt"))).send_keys(start_str)
wait.until(EC.element_to_be_clickable((By.ID, "endDt"))).send_keys(end_str)

# 면적 설정
bldgSqmEnd_input = wait.until(EC.element_to_be_clickable((By.ID, "bldgSqmEnd")))
bldgSqmEnd_input.clear()
bldgSqmEnd_input.send_keys(85)
time.sleep(1)

# 진행상태 설정
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#stat"))).click()
Select(wait.until(EC.element_to_be_clickable((By.ID, "stat")))).select_by_visible_text('-선택-')


# --- 검색 및 리스트 로딩 ---
wait.until(EC.element_to_be_clickable((By.ID, "btnSrch"))).click()
Select(wait.until(EC.element_to_be_clickable((By.ID, "dataSize_s")))).select_by_visible_text('100')
WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#lsTbody > tr.hand')))
WebDriverWait(driver, 10).until(lambda d: d.execute_script("return document.readyState") == "complete")

print("\n🔍 리스트 로딩 완료")

# --- 결과 저장용 ---
result1 = []
cols = ['사건번호', '낙찰가']

# --- 페이지별 데이터 수집 ---
while True:
    # 스크롤 끝까지 내려서 전체 tr 다 로딩되게 만들기
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

    # 새로 로딩된 tr 요소들 다시 수집
    trs = driver.find_elements(By.CSS_SELECTOR, "#lsTbody > tr.hand")
    print(f"🔵 현재 페이지 tr 개수: {len(trs)}개")

    # tr 하나하나 수집
    for idx, tr in enumerate(trs, start=1):
        try:
            tid = tr.find_element(By.CSS_SELECTOR, 'input[id^="Tid_"]').get_attribute("value")
            num = tr.find_element(By.CSS_SELECTOR, ".clear.f15.bold").text
            final_price = int(tr.find_element(By.ID, f"sucbAmt_{tid}").text.replace(",", ""))

            # 결과 저장
            result1.append([ num, final_price ])
            print(f"✅ {idx}번째 tr 추출 완료")

            time.sleep(2)

        except Exception as e:
            print(f"⚠️ {idx}번째 tr 추출 실패: {e}")

    # 다음 페이지 없으면 break
    if not move_to_next_page(driver):
        print("✅ 다음 페이지 없음, 크롤링 종료")
        break

# --- 결과 저장 ---
df = pd.DataFrame(result1, columns=cols)
df.to_csv(f"/Users/jihyun/Desktop/자료/경매/입찰결정/[{start_dt.month:02d}월{start_dt.day:02d}일-{end_dt.month:02d}월{end_dt.day:02d}일]{keyword1}_{keyword2}_낙찰가.csv", encoding="utf-8-sig", index=False)
print("✅ 파일 저장 완료")




🔍 리스트 로딩 완료
🔵 현재 페이지 tr 개수: 37개
✅ 1번째 tr 추출 완료
✅ 2번째 tr 추출 완료
⚠️ 3번째 tr 추출 실패: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="sucbAmt_2259873"]"}
  (Session info: chrome=125.0.6422.61); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x00000001099c76b8 chromedriver + 6149816
1   chromedriver                        0x00000001099becda chromedriver + 6114522
2   chromedriver                        0x000000010944bd5c chromedriver + 400732
3   chromedriver                        0x0000000109498aa5 chromedriver + 715429
4   chromedriver                        0x0000000109498bf1 chromedriver + 715761
5   chromedriver                        0x000000010948d106 chromedriver + 667910
6   chromedriver                        0x00000001094bbcdd chromedriver + 859357
7   chromedriver                        

✅ 예상 낙찰 비율 추출 성공: 100.27%
✅ 98번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 88.44%
✅ 99번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 88.44%
✅ 100번째 tr 추출 완료
✅ 다음 페이지로 이동합니다
🔵 현재 페이지 tr 개수: 100개
✅ 예상 낙찰 비율 추출 성공: 101.14%
✅ 1번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 87.24%
✅ 2번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 96.87%
✅ 3번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 84.05%
✅ 4번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 84.05%
✅ 5번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 93.06%
✅ 6번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 90.69%
✅ 7번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 90.11%
✅ 8번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 85.4%
✅ 9번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 96.28%
✅ 10번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 85.53%
✅ 11번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 85.99%
✅ 12번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 93.27%
✅ 13번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 97.45%
✅ 14번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 85.86%
✅ 15번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 90.79%
✅ 16번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 86.52%
✅ 17번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 74.51%
✅ 18번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 80.67%
✅ 19번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 95.25%
✅ 20번째 tr 추출 완료
✅ 예상 낙찰 비율 추출 성공: 87.8%
✅ 2